In [1]:
%pip install -U earthengine-api geemap folium rasterio numpy matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ee
import geemap
import geemap.foliumap as geefolium
import folium
import numpy as np
import rasterio
import matplotlib.pyplot as plt

In [3]:
# ---- SECTION 1: Setup and Environment Configuration ----
from pathlib import Path

OUTDIR = Path("ee_outputs")
OUTDIR.mkdir(parents=True, exist_ok=True)

# Optional: set EE project from environment
EE_PROJECT = "ee-milos-makes-maps" # add your own here

In [4]:
# ---- SECTION 2: Earth Engine Authentication and Initialization ----
try:
    if EE_PROJECT:
        ee.Initialize(project=EE_PROJECT)
    else:
        ee.Initialize()
except Exception:
    ee.Authenticate()
    if EE_PROJECT:
        ee.Initialize(project=EE_PROJECT)
    else:
        ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [5]:
# Quick connectivity test - should print 3
connectivity_test = ee.Number(1).add(2).getInfo()
print("connectivity test: ", connectivity_test)

connectivity test:  3


In [6]:
# ---- SECTION 3: Area of Interest (AOI) Definition ----
# 83.919067, 28.190059, 83.977776, 28.235137
xmin, ymin = 83.919067, 28.190059
xmax, ymax = 83.977776, 28.235137

region = ee.Geometry.Polygon([[
    [xmin, ymin], [xmin, ymax],
    [xmax, ymax], [xmax, ymin]
]])

In [7]:
# ========================================================================
# PART I: CLUSTERING ANALYSIS
# ========================================================================

# ---- SECTION 4: Satellite Embeddings Data Acquisition ----
embeddings_ic = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL") \
    .filterDate("2024-01-01", "2025-01-01") \
    .filterBounds(region)

embeddings_image = embeddings_ic.mosaic().clip(region).toFloat()

In [8]:
# ---- SECTION 5: Training Data Generation for Clustering ----
n_samples = 1000
training = embeddings_image.sample(
    region=region, scale = 10, numPixels = n_samples,
    seed = 100, geometries = False
)

In [11]:
# ---- SECTION 6: K-means Clustering Function ----
def get_clusters(n_clusters: int) -> ee.Image:
    clusterer = ee.Clusterer.wekaKMeans(nClusters=int(n_clusters)).train(training)
    clustered = embeddings_image.cluster(clusterer)
    return clustered

clustered_k5 = get_clusters(5)

In [ ]:
# ---- SECTION 7–8: Interactive Visualization (saved to HTML) ----
# Kelly's 22 colors (distinct). Use first N for discrete classes.
kelly22 = [
    "#F3C300","#875692","#F38400","#A1CAF1","#BE0032","#C2B280","#848482","#008856",
    "#E68FAC","#0067A5","#F99379","#604E97","#F6A600","#B3446C","#DCD300","#882D17",
    "#8DB600","#654522","#E25822","#2B3D26","#F2F3F4","#222222"
]

vis_k5 = dict(min = 0, max = 4, palette = kelly22[:5]) # MUST BE =
m = geefolium.Map(location = [ymin, xmin], zoom_start = 13, control_scale = True)
m.add_basemap("SATELLITE")
m.addLayer(clustered_k5.toInt(), vis_k5, "K=5 clusters")
m.fit_bounds([[ymin, xmin]], [[ymax, xmax]])
map_path = OUTDIR / "map_kp.html"
m.save(str(map_path))

In [14]:
# %%
# ---- SECTION 9–10: Export K = 3, 5, 10 locally (GeoTIFF) ----
def export_cluster_local(k: int, outdir: Path = OUTDIR):
    img = get_clusters(k).toInt().clip(region)
    tif = outdir / f"clusters_k{k}.tif"
    geemap.ee_export_image(img, str(tif), scale = 10, region = region, file_per_band=False)
    return(tif)

cluster_values = [3, 5, 10]
cluster_paths = [export_cluster_local(k) for k in cluster_values]

Generating URL ...
Please wait ...
Data downloaded to d:\python-scripts\ee_outputs\clusters_k3.tif
Generating URL ...
Please wait ...
Data downloaded to d:\python-scripts\ee_outputs\clusters_k5.tif
Generating URL ...
Please wait ...
Data downloaded to d:\python-scripts\ee_outputs\clusters_k10.tif


In [16]:
# ---- SECTION 12: Multi-panel Visualization of clusters ----
def read_single_band(path: Path):
    with rasterio.open(path) as src:
        arr = src.read(1)
        extent = (src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top)
        transform = src.transform
        crs = src.crs
    return arr, extent, transform, crs

arr5, extent5, transform5, crs5 = read_single_band(OUTDIR / "clusters_k5.tif")
arr3, _, _, _ = read_single_band(OUTDIR / "clusters_k3.tif")
arr10, _, _, _ = read_single_band(OUTDIR / "clusters_k10.tif")

In [18]:
fig, axs = plt.subplots(1, 3, figsize=(12,4), constrained_layout=True)
for ax, (arr, title, k) in zip(
    axs,
    [(arr3, "K = 3", 3), (arr5, "K = 5", 5), (arr10, "K = 10", 10)]
):
    from matplotlib.colors import ListedColormap, BoundaryNorm
    cmap = ListedColormap(kelly22[:k])
    bounds = np.arange(-0.5, k + 0.5, 1)
    norm = BoundaryNorm(bounds, cmap.N)
    im = ax.imshow(arr, cmap=cmap, norm=norm)
    ax.set_title(title)
    ax.set_xticks([]); ax.set_yticks([])

panel_png = OUTDIR / "clusters_panel.png"
plt.savefig(panel_png, dpi = 300, bbox_inches = "tight")
plt.close(fig)


In [19]:
# ========================================================================
# PART II: CHANGE DETECTION ANALYSIS (computed server-side in EE)
# ========================================================================

# ---- SECTION 13–14: Multi-temporal Data + Exports (local) ----
def get_satellite_embeddings(y0: int, y1: int) -> ee.Image:
    return(ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")
           .filterDate(f"{y0}-01-01", f"{y1}-01-01")
           .filterBounds(region)
           .mosaic()
           .clip(region)
           .toFloat()
    )

emb_2018 = get_satellite_embeddings(2018, 2019)
emb_2024 = get_satellite_embeddings(2024, 2025)

In [20]:
# Mean absolute difference across bands (single-band image)
mad_img = emb_2024.subtract(emb_2018).abs().reduce(ee.Reducer.mean())
mad_tif = OUTDIR / "mean_absolute_difference.tif"
geemap.ee_export_image(mad_img, filename=str(mad_tif), scale=10, region=region)

Generating URL ...
Please wait ...
Data downloaded to d:\python-scripts\ee_outputs\mean_absolute_difference.tif


In [21]:
# Cosine similarity across 64-D embeddings (single-band image)
a = emb_2018.toArray()
b = emb_2024.toArray()

# dot(a,b)
dot = a.multiply(b).arrayReduce(ee.Reducer.sum(), [0]).arrayGet([0])
na = a.multiply(a).arrayReduce(ee.Reducer.sum(), [0]).arrayGet([0]).sqrt()
nb = b.multiply(b).arrayReduce(ee.Reducer.sum(), [0]).arrayGet([0]).sqrt()

# Avoid divide-by-zero, clamp to [-1, 1], and name the band
den = na.multiply(nb)
cos = dot.divide(den).clamp(-1, 1).updateMask(den.neq(0)).rename('cosine')

# Export (use positional args or ee_image= depending on your geemap version)
cos_tif = OUTDIR / "cosine_2018_2024.tif"
geemap.ee_export_image(cos, str(cos_tif), scale = 10, region=region)

Generating URL ...
Please wait ...
Data downloaded to d:\python-scripts\ee_outputs\cosine_2018_2024.tif


In [24]:
# ---- SECTION 16 & 18: Local Visualization of change maps ----
def plot_single_band_tif(
        tif_path: Path, title: str, cmap="magma",
        vmin=None, vmax=None, outfile: Path = None
):
    with rasterio.open(tif_path) as src:
        arr = src.read(1)
        extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]
    fig = plt.figure(figsize=(7.5, 6))
    plt.imshow(arr, cmap=cmap, vmin=vmin, vmax=vmax)
    plt.title(title)
    plt.xticks([]), plt.yticks([])
    cbar = plt.colorbar()
    if outfile is not None:
        plt.savefig(outfile, dpi=600, bbox_inches="tight")
    plt.close(fig)

mad_png = OUTDIR / "mean_absolute_difference.png"
plot_single_band_tif(
    mad_tif, title = "Mean Absolute Difference (2018-2024)",
    cmap="magma", outfile=mad_png
)

cos_png = OUTDIR / "cosine_change_heatmap.png"
plot_single_band_tif(
    cos_tif, title = "Cosine Similarity (2018-2024)\nLower = more change",
    cmap="magma", vmin = -1, vmax = 1, outfile=cos_png
)
